## Solution 2
### Approach
1) Select a few independent variables (across sectors)

2) Map additional data for plotting geospatial visualizations

3) Create widgets for controlling visualizations

4) Create dynamic visualizations

In [1]:
%matplotlib inline

#For Visualizations & data wrangling
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import math

from scipy import stats
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

#For Widgets
from matplotlib.widgets import Slider, Button 
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


#For geoplots
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable

print("*"*100)

****************************************************************************************************


In [2]:
#Import files into datasets
WDIData = pd.read_csv("/Users/sagarpardeshi/Downloads/WDI_csv/WDIData.csv")
WDISeries = pd.read_csv("/Users/sagarpardeshi/Downloads/WDI_csv/WDISeries.csv")

#Merge the datasets of Series and WDIData
WDI_Data = pd.merge(WDIData, WDISeries[['Series Code','Topic']], 
                    left_on = 'Indicator Code',
                    right_on ='Series Code', how ='left')

#Create independent datasets of each indicator

# WDIData_WB_Debt = WDI_Data[WDI_Data['Topic']=='World Bank, International Debt Statistics.']
# WDIData_PS_GovFin = WDI_Data[WDI_Data['Topic']=='Public Sector: Government finance']
# WDIData_PST_Trade = WDI_Data[WDI_Data['Topic']=='Private Sector & Trade: Total merchandise trade']
# WDIData_Gender_Agency = WDI_Data[WDI_Data['Topic']=='Gender: Agency']
# WDIData_EPD_GNI = WDI_Data[WDI_Data['Topic']=='Economic Policy & Debt: National accounts: Atlas GNI & GNI per capita!']
WDI = WDI_Data.drop(['Indicator Name','Indicator Code','Series Code'], axis = 1)
WDI = WDI.replace(np.nan,0,regex=True)
WDI = WDI.loc[:, ~WDI.columns.str.contains('^Unnamed')]
df = pd.melt(WDI, id_vars=["Country Code","Country Name","Topic"],var_name="Year",value_name="Values")
df['Year'] = pd.to_numeric(df['Year'])
print("*"*100)

****************************************************************************************************


### Following dependent variables are selected to identify key features:
- World Bank, International Debt Statistics
- Public Sector: Government finance
- Private Sector & Trade: Total merchandise trade
- Gender: Agency
- Economic Policy & Debt: National accounts: Atlas GNI & GNI per capita!


### Create plot functions to make generic use

In [3]:
def plot_maps(df, top_label):
    fig, ax = plt.subplots(1, 1)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    df.plot(ax = ax, column = df['Values'], legend = True,cax = cax,cmap='OrRd',label=top_label,
                  # edgecolor = 'black',
                  # linewidth = 0.5,
                   missing_kwds={
                "color": "lightgrey",
                "edgecolor": "red",
                "hatch": "/--",
                "label": "Values missing for the zone",
            },
                   legend_kwds={'label': top_label,'orientation': "vertical"})
    fig = plt.gcf()
    fig.set_figheight(12)
    fig.set_figwidth(10)
print("*"*100)

****************************************************************************************************


### Create widgets and call plot functions

In [4]:
def f2(top,year):
    year = year
    top = top
    df_filtered = df[df['Topic'] == top]
    df_filtered = df_filtered[df_filtered['Year']==year]
    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    world_cleaned = pd.read_csv("/Users/sagarpardeshi/Downloads/WDI_csv/world.csv")
    world_map = pd.merge(world, world_cleaned[['name','CC']], 
                    on = 'name',
                    how ='left')
    world_map = pd.merge(world_map, df_filtered, 
                     left_on = 'CC', right_on = 'Country Code', how = 'left')
    plot_maps(world_map,top)

interact(f2, top=widgets.Dropdown(options = 
                    ['World Bank, International Debt Statistics.'
                     ,'Public Sector: Government finance'
                     ,'Private Sector & Trade: Total merchandise trade'
                     ,'Gender: Agency']              ,
                    value = 'Public Sector: Government finance',
                    description = 'Indicator'),
             year=widgets.IntSlider(min=min(df.Year),
                                max=max(df.Year), 
                                step=1,
                                description='Year:',
                                disabled=False,
                                continuous_update=False,
                                orientation='horizontal',
                                readout=True,
                                readout_format='d',
                                value=2014))
print("*"*100)

interactive(children=(Dropdown(description='Indicator', index=1, options=('World Bank, International Debt Stat…

****************************************************************************************************


### Create animations for the visualizations 

In [5]:

# import random
# from itertools import count
# from matplotlib.animation import FuncAnimation

# fig, ax = plt.subplots(1, 1)
# plt.figure(figsize=((10,8)))
# x_vals = [0]
# y_vals = [4]
# index = count()
# plt.style.use('fivethirtyeight')

# def animate(i):
#     plt.cla()
#     print(x_vals)
#     print(x_vals)
#     plt.tight_layout()
#     plt.figure(figsize=((10,8)))
#     plt.plot(x_vals,y_vals)
#     x_vals.append(next(index))
#     y_vals.append(random.randint(1,5))



    
# ani = FuncAnimation(plt.gcf(),animate, interval = 2000)
# ani

In [6]:
from IPython.display import clear_output
yr = 2000
def animate(top, yr):
    for i in range(yr,max(df['Year'])-1):
        print("Year is %i" %yr)
        yr = yr+1
        df_filtered_2 = df[df['Year']==yr]
        data = df_filtered_2[df_filtered_2['Topic'] == top].sort_values('Values',ascending = False).head(10)
        y_vals = data['Values']
        x_vals = data['Country Name']
        plt.xlabel(top)
        plt.gca().invert_yaxis()
        plt.ylabel("Country")
        plt.title("Top Countries - %i" %yr)
        fig = plt.gcf()
        fig.set_size_inches(11,7)
        plt.barh(x_vals,y_vals, color ='maroon',align = 'center')
        for j in plt.gca().patches:
            plt.text(j.get_width()+0.2, j.get_y()+0.5,
                     str(round((j.get_width()), 2)),
                     fontsize = 10, fontweight ='bold',
                     color ='grey')
        plt.pause(5)  
        plt.cla()
        clear_output(wait=True)     
    plt.show()

interact(animate, top=widgets.Dropdown(options = 
                    ['World Bank, International Debt Statistics.'
                     ,'Public Sector: Government finance'
                     ,'Private Sector & Trade: Total merchandise trade'
                     ,'Gender: Agency']              ,
                    value = 'World Bank, International Debt Statistics.',
                    description = 'Indicator'), yr=widgets.IntSlider(min=min(df.Year),
                                max=max(df.Year), 
                                step=1,
                                description='Year:',
                                disabled=False,
                                continuous_update=False,
                                orientation='horizontal',
                                readout=True,
                                readout_format='d',
                                value=1993))

interactive(children=(Dropdown(description='Indicator', options=('World Bank, International Debt Statistics.',…

<function __main__.animate(top, yr)>